In [ ]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import g_key

In [ ]:
df = pd.read_csv("Zomato_Restaurants_All_Consolidated-Types.csv")
df

In [ ]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)